In [ ]:
import imageio as iio
from matplotlib import pyplot as plt
import numpy as np
import cv2
import imutils
from PIL import Image, ImageDraw
import itertools
from scipy.ndimage.filters import gaussian_filter as __gaussian_filter
from scipy.ndimage.filters import convolve as __convolve
from scipy.ndimage.filters import correlate as __correlate
from scipy.fftpack import fftshift as __fftshift

import nibabel as nib
import pandas as pd

<ipython-input-42-0882c34063c9>:8: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter as __gaussian_filter
<ipython-input-42-0882c34063c9>:9: DeprecationWarning: Please use `convolve` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import convolve as __convolve
<ipython-input-42-0882c34063c9>:10: DeprecationWarning: Please use `correlate` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import correlate as __correlate


In [ ]:
def process_image(img):



    def calculate_image_centroid(image):
        # Convert the image to grayscale
        grayscale_image = np.array(image.convert("L"))


        # Get the height and width of the image
        height, width = grayscale_image.shape

        # Initialize variables for weighted sums and total intensity
        weighted_sum_x = 0
        weighted_sum_y = 0
        total_intensity = 0

        # Iterate through each pixel
        for y in range(height):
            for x in range(width):
                intensity = grayscale_image[y, x]

                # Calculate the weighted sum of pixel coordinates
                weighted_sum_x += intensity * x
                weighted_sum_y += intensity * y

                # Calculate the sum of pixel intensities
                total_intensity += intensity

        # Calculate the centroid coordinates
        centroid_x = weighted_sum_x / total_intensity
        centroid_y = weighted_sum_y / total_intensity

        return centroid_x, centroid_y

    # Convert the image to PIL Image
    pil_image = Image.fromarray(img)

    centroid = calculate_image_centroid(pil_image)

    # cambiar el centro de la imagen
    def change_image_center(img, new_center):
        height, width = img.shape[:2]
        center_x, center_y = new_center
        current_center_point = (width // 2, height // 2)

        # Calculate the pixel shifts for padding
        if current_center_point[0] == center_x and current_center_point[1] == center_y:
          padded_img = img
        if current_center_point[0] > center_x and current_center_point[1]> center_y:
          padded_img = cv2.copyMakeBorder(img, int(height-2*center_y), 0, 0,0, cv2.BORDER_CONSTANT, value=(0, 0, 0))
        if current_center_point[0] == center_x and current_center_point[1]> center_y:
          padded_img = cv2.copyMakeBorder(img, int(height-2*center_y), 0, int(width-2*center_x),0, cv2.BORDER_CONSTANT, value=(0, 0, 0))
        if current_center_point[0] < center_x and current_center_point[1]>center_y:
          padded_img = cv2.copyMakeBorder(img, int(height-2*center_y), 0, 0, int(width-2*(width-center_x)), cv2.BORDER_CONSTANT, value=(0, 0, 0))
        if current_center_point[0] < center_x and current_center_point[1]==center_y:
          padded_img = cv2.copyMakeBorder(img, 0, 0, 0, int(width-2*(width-center_x)), cv2.BORDER_CONSTANT, value=(0, 0, 0))
        if current_center_point[0] > center_x and current_center_point[1] < center_y:
          padded_img = cv2.copyMakeBorder(img, 0, int(height-2*(height-center_y)), int(width-2*center_x), 0, cv2.BORDER_CONSTANT, value=(0, 0, 0))
        if current_center_point[0] > center_x and current_center_point[1] == center_y:
          padded_img = cv2.copyMakeBorder(img, 0, 0, int(width-2*center_x), 0, cv2.BORDER_CONSTANT, value=(0, 0, 0))
        if current_center_point[0] < center_x and current_center_point[1] < center_y:
          padded_img = cv2.copyMakeBorder(img, 0, int(height-2*(height-center_y)), 0, int(width-2*(width-center_x)), cv2.BORDER_CONSTANT, value=(0, 0, 0))
        if current_center_point[0] == center_x and current_center_point[1] < center_y:
          padded_img = cv2.copyMakeBorder(img, 0, int(height-2*(height-center_y)), 0, 0, cv2.BORDER_CONSTANT, value=(0, 0, 0))

        return padded_img

    img2a = change_image_center(img, centroid)

    # Agrego margen
    m=5
    img2 = cv2.copyMakeBorder(img2a, m, m, m, m, cv2.BORDER_CONSTANT, value=(0, 0, 0))

    def rotate_and_flip_image(img, angles, centers):
        rotated_and_flipped_images = []

        for angle, center in itertools.product(angles, centers):
            # change_image_center
            padded_img=change_image_center(img, center)
            height, width = padded_img.shape[:2]
            centerX, centerY = (width // 2, height // 2)
            # get rotation matrix with the specified center
            M = cv2.getRotationMatrix2D((centerX,centerY), angle, 1.0)
            rotated = cv2.warpAffine(padded_img, M, (width, height))


            flipped = np.fliplr(rotated)

            rotated_and_flipped_images.append((rotated, flipped))

        return rotated_and_flipped_images

    angles = [0, 0.5, -0.5, 1, -1, 1.5, -1.5, 2, -2, 2.5, -2.5, 3, -3, 3.5, -3.5, 4, -4, 4.5, -4.5, 5, -5]  # Example angles, you can modify this list as per your requirements
    height, width = img2.shape[:2]
    centerx, centery = (width // 2, height // 2)
    #centerx=cropped_image2.shape[1]/2
    #centery=cropped_image2.shape[0]/2
    b=1 #cuanto mover el centro?
    d=2
    centers = [(centerx,centery), (centerx+b,centery), (centerx-b,centery), (centerx,centery-b), (centerx,centery+b),(centerx-b,centery+b),(centerx-b,centery-b),(centerx+b,centery+b),(centerx+b,centery-b),(centerx+d,centery), (centerx-d,centery), (centerx,centery-d), (centerx,centery+d),(centerx-d,centery+d),(centerx-d,centery-d),(centerx+d,centery+d),(centerx+d,centery-d)]
    result = rotate_and_flip_image(img2, angles, centers)

    def __is_grey_scale(image):
        """
        Check whether an image is grayscale or not
        :param image: image data
        :return: True if the image is grayscale, False otherwise
        """
        if len(image.shape) > 2:
            return False
        else:
            return True

    def gms(reference, query, pool='mean'):
        """
        Computes the Gradient Magnitude Similarity Mean (GMSM) and the Gradient Magnitude Similarity Deviation (GMSD)
        It also calculates the quality_map: local quality map of the distorted image and gradient map of each image
        GMSD is an implementation of the following algorithm: Wufeng Xue, Lei Zhang, Xuanqin Mou, and Alan C. Bovik,
        "Gradient Magnitude Similarity Deviation: A Highly Efficient Perceptual Image Quality Index",
        This code is a translation of the matlab code that can be downloaded here:
        http://www.comp.polyu.edu.hk/~cslzhang/IQA/GMSD/GMSD.htm
        :param reference: the reference image (grayscale image, double type, 0~255)
        :param query: the distorted image (grayscale image, double type, 0~255)
        :param pool: pooling strategy ('mean' for GMSM, 'std' for GMSD)
        :return: GMSM value if pool='mean', GMSD value if pool='std'
        """

        if not(__is_grey_scale(query)):
            reference = cv2.cvtColor(reference, cv2.COLOR_BGR2GRAY) # convert to gray scale
            query = cv2.cvtColor(query, cv2.COLOR_BGR2GRAY)

        (C, Down_step) = (170, 2)

        hx = np.array([[1.0, 0.0, -1.0], [1.0, 0.0, -1.0], [1.0, 0.0, -1.0]]) / 3
        hy = hx.transpose()
        # these are the Prewitt filters along horizontal (x) and vertical (y) directions

        (ref, dist) = (reference.astype('double'), query.astype('double'))

        aveKernel = np.array([[1.0, 1.0], [1.0, 1.0]]) / 4  # averaging filter

        averef = __convolve(ref, aveKernel, mode='constant', cval=0.0)
        avedist = __convolve(dist, aveKernel, mode='constant', cval=0.0)

        ref = averef[::Down_step, ::Down_step]
        dist = avedist[::Down_step, ::Down_step]

        Ixref = __convolve(ref, hx, mode='constant', cval=0.0)
        Iyref = __convolve(ref, hy, mode='constant', cval=0.0)
        gradientMap1 = np.sqrt(Ixref**2 + Iyref**2)

        Ixdist = __convolve(dist, hx, mode='constant', cval=0.0)
        Iydist = __convolve(dist, hy, mode='constant', cval=0.0)
        gradientMap2 = np.sqrt(Ixdist**2 + Iydist**2)

        quality_map = (2 * gradientMap1 * gradientMap2 + C) / (gradientMap1**2 + gradientMap2**2 + C)

        if pool == 'mean':
            return np.mean(quality_map)
        elif pool == 'std':
            return np.std(quality_map)

    # n es la cantidad de ángulos que se prueban
    n=len(angles)
    m=len(centers)

    def similaridad(lista):
        #img = cv2.imread(image_path)  # Read the image
        similaridad = []

        for i in range(n*m):
            # Aplico medidad de similitud
            med=gms(lista[i][0] ,lista[i][1], pool='std')


            # Append rotated and flipped image to the list
            similaridad.append(med)

        return similaridad

    sim = similaridad(result)
    sim

    # Calculamos el mínimo

    minimum_value = min(sim)
    minimum_index = sim.index(minimum_value)
    print(minimum_value)
    print(minimum_index)

    v=(minimum_index,minimum_value)

    return v


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# List of image paths CONTROL
image_paths = ["/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0001_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0002_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0010_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0011_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0013_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0018_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0019_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0020_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0026_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0030_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0032_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0033_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0034_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0044_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0058_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0062_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0064_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0065_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0068_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0069_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0070_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0071_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0072_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0074_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0075_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0078_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0083_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0085_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0086_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0096_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0106_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0109_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0110_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0112_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0113_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0114_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0116_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0130_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0133_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0135_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0138_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0139_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0146_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0157_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0165_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0169_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0170_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0176_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0177_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0180_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0181_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0186_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0188_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0195_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0197_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0199_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0200_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0201_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0203_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0204_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0206_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0207_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0208_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0212_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0216_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0220_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0221_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0228_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0229_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0234_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0237_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0241_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0244_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0254_MR1_mpr_n6_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0255_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0256_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0259_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0260_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0262_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0266_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0270_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0271_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0274_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0279_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0280_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0284_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0289_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0292_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0293_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0299_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0301_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0303_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0309_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0317_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0322_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0323_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0326_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0330_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0332_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0337_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0338_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0341_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0342_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0343_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0345_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0354_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0355_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0356_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0357_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0358_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0363_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0365_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0366_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0367_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0369_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0371_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0372_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0378_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0381_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0398_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0401_MR1_mpr_n5_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0404_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0422_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0423_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0426_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0428_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0433_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0438_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0443_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0445_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0446_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0449_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0455_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0456_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0457_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               ]




# List to store the processed matrix
processed_matrixs_CONTROL = []

# Process each image
for image_path in image_paths:
    # Open the image
    img = nib.load(image_path)

    # Access the image data as a NumPy array
    image_data = img.get_fdata()

    image_data = image_data.squeeze()


    # elijo slice para ver, por ejemplo la 90 [43, 48, 53, 58, 63,68,73,78,83,88,93,98,103,108,113,118,123,128,133]
    processed_vectors_CONTROL = []
    for i in [43, 48, 53, 58, 63,68,73,78,83,88,93,98,103,108,113,118,123,128,133]:
        img90=image_data[:,:,i]
        image2 = imutils.rotate_bound(img90, angle=-90)

    # image2 = np.diagonal(image_data, offset=0, axis1=1, axis2=2).T #oblicua

        image = cv2.convertScaleAbs(image2, alpha=0.03)

    # Process the image
        processed_vector_CONTROL = process_image(image)

        processed_vectors_CONTROL.append(processed_vector_CONTROL)

    processed_matrix_CONTROL = processed_vectors_CONTROL

    processed_matrixs_CONTROL.append(processed_matrix_CONTROL)

Se han truncado las últimas 5000 líneas del flujo de salida.
0.09063625936194977
28
0.09283855066692126
28
0.09274847117479663
29
0.10015755987058401
41
0.09284820731871148
113
0.08870488931937366
25
0.09427603442255193
46
0.08225810367387157
48
0.0825138970057003
31
0.08055624324704742
45
0.07885772269180903
58
0.09063111045376326
166
0.09591926137160914
53
0.11084540002103273
39
0.1205843667271125
28
0.11522933775694782
28
0.10495834115029837
28
0.10494623639846741
28
0.10552165280428803
33
0.1067360339720762
62
0.10541210995647327
28
0.11778709671736075
97
0.11331782101994319
56
0.11523610194868568
31
0.12024750779756488
131
0.09763731438393343
75
0.10306574360303984
63
0.10115625747736595
59
0.08231328364792283
25
0.07440374002745162
65
0.07874851341503067
28
0.08078574149081777
62
0.08623254063417948
28
0.09181001996733655
45
0.09220909006002069
96
0.09135642410334376
25
0.08637968030051622
63
0.08851249643299587
28
0.08480229341022501
28
0.08016589837604138
22
0.0912901864005825


In [ ]:
processed_matrixs_CONTROL

[[(28, 0.08737715449863268),
  (29, 0.09699739628796598),
  (28, 0.09618939835885078),
  (39, 0.10496480445048494),
  (62, 0.10077517554964738),
  (28, 0.08875250045582544),
  (24, 0.09978826473998477),
  (42, 0.09791959878072574),
  (59, 0.08923151492251774),
  (56, 0.09384933162311664),
  (22, 0.10427294770441548),
  (24, 0.09663205411590137),
  (42, 0.0860619581377442),
  (83, 0.09366889869915225),
  (93, 0.08762697532459182),
  (113, 0.08790381800705464),
  (127, 0.08834906828702305),
  (290, 0.08169396720812386),
  (46, 0.07605870068897744)],
 [(28, 0.06194645032991042),
  (62, 0.07281292696060801),
  (28, 0.08372203601914902),
  (28, 0.08644269570809042),
  (29, 0.07914332102118571),
  (29, 0.08226245244959297),
  (62, 0.07458931892054893),
  (25, 0.08221928401619553),
  (58, 0.07981391469369901),
  (63, 0.0894496233990722),
  (62, 0.08090135144357005),
  (29, 0.08830557034267404),
  (31, 0.08586280465000458),
  (48, 0.09694187086833078),
  (49, 0.08273800850693927),
  (33, 0.082

In [ ]:
rows = []
for sublist in processed_matrixs_CONTROL:
    row_dict = {'Column1': sublist[0][1], 'Column2': sublist[1][1], 'Column3': sublist[2][1], 'Column4': sublist[3][1],'Column5': sublist[4][1], 'Column6': sublist[5][1],'Column7': sublist[6][1], 'Column8': sublist[7][1],'Column9': sublist[8][1], 'Column10': sublist[9][1],'Column11': sublist[10][1], 'Column12': sublist[11][1],'Column13': sublist[12][1], 'Column14': sublist[13][1],'Column15': sublist[14][1], 'Column16': sublist[15][1],'Column17': sublist[16][1], 'Column18': sublist[17][1],'Column19': sublist[18][1]}
    rows.append(row_dict)

# Convertir la lista de diccionarios a un DataFrame
control = pd.DataFrame(rows)

# Mostrar el DataFrame
print(control)


      Column1   Column2   Column3   Column4   Column5   Column6   Column7  \
0    0.087377  0.096997  0.096189  0.104965  0.100775  0.088753  0.099788   
1    0.061946  0.072813  0.083722  0.086443  0.079143  0.082262  0.074589   
2    0.080250  0.085513  0.094510  0.096007  0.095845  0.098865  0.109415   
3    0.068644  0.084979  0.097191  0.098029  0.099798  0.086103  0.096863   
4    0.090631  0.095919  0.110845  0.120584  0.115229  0.104958  0.104946   
..        ...       ...       ...       ...       ...       ...       ...   
130  0.083246  0.084114  0.096388  0.100567  0.115346  0.098139  0.094783   
131  0.082536  0.086016  0.102183  0.095406  0.086447  0.082468  0.083709   
132  0.076048  0.087315  0.100049  0.092917  0.093139  0.087059  0.085475   
133  0.064577  0.093716  0.115846  0.106634  0.102228  0.113261  0.111470   
134  0.079180  0.078532  0.096520  0.094886  0.103916  0.088027  0.093910   

      Column8   Column9  Column10  Column11  Column12  Column13  Column14  

In [ ]:
# List of image paths AD
image_paths = ["/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0003_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0015_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0016_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0021_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0022_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0023_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0028_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0031_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0035_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0039_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0041_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0042_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0046_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0052_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0053_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0056_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0060_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0066_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0067_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0073_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0082_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0084_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0094_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0098_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0115_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0120_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0122_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0123_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0124_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0134_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0137_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0142_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0143_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0155_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0158_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0161_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0164_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0166_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0179_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0184_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0185_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0205_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0210_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0217_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0223_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0226_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0233_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0238_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0240_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0243_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0247_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0263_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0267_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0268_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0269_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0272_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0273_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0278_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0286_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0287_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0288_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0290_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0291_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0298_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0300_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0304_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0307_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0308_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0312_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0315_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0316_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0329_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0335_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0339_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0351_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0352_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0362_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0373_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0374_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0380_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0382_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0388_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0390_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0399_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0400_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0402_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0405_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0411_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0418_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0424_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0425_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0430_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0432_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0440_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0441_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0447_MR1_mpr_n4_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0451_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0452_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0453_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               "/content/drive/Shareddrives/Team-Famaf/Códigos/Base/OAS1_0454_MR1_mpr_n3_anon_111_t88_masked_gfc.hdr",
               ]




# List to store the processed vectors
processed_matrixs_AD = []

# Process each image
for image_path in image_paths:
    # Open the image
    img = nib.load(image_path)

    # Access the image data as a NumPy array
    image_data = img.get_fdata()

    image_data = image_data.squeeze()
    # elijo slice para ver, por ejemplo la 90
    processed_vectors_AD = []
    for i in [43, 48, 53, 58, 63,68,73,78,83,88,93,98,103,108,113,118,123,128,133]:
        img90=image_data[:,:,i]
        image2 = imutils.rotate_bound(img90, angle=-90)

       # image2 = np.diagonal(image_data, offset=0, axis1=1, axis2=2).T  # oblicua

        image = cv2.convertScaleAbs(image2, alpha=0.03)

    # Process the image
        processed_vector_AD = process_image(image)

        processed_vectors_AD.append(processed_vector_AD)

    processed_matrix_AD = processed_vectors_AD

    processed_matrixs_AD.append(processed_matrix_AD)

0.08308844335907044
62
0.09371993067963821
22
0.0995777457823247
30
0.09535402514673703
28
0.09884046745090778
63
0.11021886997701263
126
0.10917248282823686
59
0.11160830118775193
86
0.1048013771565051
59
0.10127726638925813
93
0.11141030912499042
58
0.10509925205940873
93
0.1014646051288223
25
0.09634082475157436
41
0.10298028048084482
147
0.10070481364218813
92
0.0995347694948694
188
0.08239232379759948
82
0.079050166788645
52
0.07750986246665287
59
0.08293181729063304
52
0.09320853946742728
25
0.0935878662687983
28
0.0949456621134878
96
0.0924884322021515
29
0.09343069798878754
25
0.09614093512203105
25
0.09586509270778375
25
0.10600691981872315
28
0.09789699906262159
31
0.09457512167670973
49
0.11402208310622783
49
0.11261431633711648
83
0.1065179910626417
101
0.10577948883723053
33
0.0997375476081698
46
0.08599584979398374
165
0.0862974775303139
194
0.07754374154849611
28
0.07846059320053174
28
0.09225468670516947
28
0.09365269097576366
45
0.0883125572216237
25
0.0804906831489325

In [ ]:
processed_matrixs_AD

[[(62, 0.08308844335907044),
  (22, 0.09371993067963821),
  (30, 0.0995777457823247),
  (28, 0.09535402514673703),
  (63, 0.09884046745090778),
  (126, 0.11021886997701263),
  (59, 0.10917248282823686),
  (86, 0.11160830118775193),
  (59, 0.1048013771565051),
  (93, 0.10127726638925813),
  (58, 0.11141030912499042),
  (93, 0.10509925205940873),
  (25, 0.1014646051288223),
  (41, 0.09634082475157436),
  (147, 0.10298028048084482),
  (92, 0.10070481364218813),
  (188, 0.0995347694948694),
  (82, 0.08239232379759948),
  (52, 0.079050166788645)],
 [(59, 0.07750986246665287),
  (52, 0.08293181729063304),
  (25, 0.09320853946742728),
  (28, 0.0935878662687983),
  (96, 0.0949456621134878),
  (29, 0.0924884322021515),
  (25, 0.09343069798878754),
  (25, 0.09614093512203105),
  (25, 0.09586509270778375),
  (28, 0.10600691981872315),
  (31, 0.09789699906262159),
  (49, 0.09457512167670973),
  (49, 0.11402208310622783),
  (83, 0.11261431633711648),
  (101, 0.1065179910626417),
  (33, 0.1057794888

In [ ]:
rows2 = []
for sublist in processed_matrixs_AD:
    row2_dict = {'Column1': sublist[0][1], 'Column2': sublist[1][1], 'Column3': sublist[2][1], 'Column4': sublist[3][1],'Column5': sublist[4][1], 'Column6': sublist[5][1],'Column7': sublist[6][1], 'Column8': sublist[7][1],'Column9': sublist[8][1], 'Column10': sublist[9][1],'Column11': sublist[10][1], 'Column12': sublist[11][1],'Column13': sublist[12][1], 'Column14': sublist[13][1],'Column15': sublist[14][1], 'Column16': sublist[15][1],'Column17': sublist[16][1], 'Column18': sublist[17][1],'Column19': sublist[18][1]}
    rows2.append(row2_dict)

# Convertir la lista de diccionarios a un DataFrame
ad = pd.DataFrame(rows2)

# Mostrar el DataFrame
print(ad)

     Column1   Column2   Column3   Column4   Column5   Column6   Column7  \
0   0.083088  0.093720  0.099578  0.095354  0.098840  0.110219  0.109172   
1   0.077510  0.082932  0.093209  0.093588  0.094946  0.092488  0.093431   
2   0.077544  0.078461  0.092255  0.093653  0.088313  0.080491  0.088851   
3   0.071994  0.081122  0.098202  0.097826  0.107066  0.104868  0.109535   
4   0.073722  0.082428  0.086396  0.087121  0.086255  0.083769  0.085687   
..       ...       ...       ...       ...       ...       ...       ...   
95  0.085566  0.096529  0.099211  0.095986  0.096536  0.098734  0.094929   
96  0.084483  0.092762  0.095791  0.100744  0.100209  0.098450  0.103711   
97  0.082177  0.077720  0.104598  0.102562  0.101650  0.096937  0.091523   
98  0.080408  0.082949  0.097866  0.103516  0.095209  0.085177  0.083870   
99  0.084355  0.076069  0.090115  0.093927  0.083589  0.083972  0.086377   

     Column8   Column9  Column10  Column11  Column12  Column13  Column14  \
0   0.11160

In [ ]:
from google.colab import files
# Export to CSV
control.to_csv('controla.csv')
# Download the file
files.download('controla.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Export to CSV
ad.to_csv('ada.csv')
# Download the file
files.download('ada.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>